В качестве домашнего задания вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить 4 модели которые будут предсказывать целевую переменную <b>RainTomorrow</b> с помощью:

   1. логистической регрессии [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
   
   2. метода ближайших соседей [sklearn.neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
 
   3. Байесовского классификатора [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
   
   4. логистической регрессии реализованной самостоятельно

Затем следует сравнить результаты моделей (по качеству и времени выполнения) и сделать вывод о том, какая модель и с какими параметрами даёт лучшие результаты.

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

In [608]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
%matplotlib notebook
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [609]:
X = pd.read_csv('weather.csv')


In [610]:
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})

In [611]:
del X['RainTomorrow']
del X['Unnamed: 0']

### Осмотрим данные

In [612]:
X.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,141556.000000,141871.000000,140787.000000,81350.000000,74377.000000,132923.000000,140845.000000,139563.000000,140419.000000,138583.000000,128179.000000,128212.000000,88536.000000,85099.000000,141289.000000,139467.000000
mean,12.186400,23.226784,2.349974,5.469824,7.624853,39.984292,14.001988,18.637576,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.987509,21.687235
std,6.403283,7.117618,8.465173,4.188537,3.781525,13.588801,8.893337,8.803345,19.051293,20.797772,7.105476,7.036677,2.887016,2.720633,6.492838,6.937594
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000
25%,7.600000,17.900000,0.000000,2.600000,4.900000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.900000,1010.400000,1.000000,2.000000,12.300000,16.600000
50%,12.000000,22.600000,0.000000,4.800000,8.500000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000
75%,16.800000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000,21.600000,26.400000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000


In [613]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

### Видим что присутсвуют и не численные типы

### Проверим на процент пропусков в каждом столбце

In [614]:
(X.isna().sum() / len(X)).round(4) * 100

Date              0.00
Location          0.00
MinTemp           0.45
MaxTemp           0.23
Rainfall          0.99
Evaporation      42.79
Sunshine         47.69
WindGustDir       6.56
WindGustSpeed     6.52
WindDir9am        7.04
WindDir3pm        2.66
WindSpeed9am      0.95
WindSpeed3pm      1.85
Humidity9am       1.25
Humidity3pm       2.54
Pressure9am       9.86
Pressure3pm       9.83
Cloud9am         37.74
Cloud3pm         40.15
Temp9am           0.64
Temp3pm           1.92
RainToday         0.99
dtype: float64

### В большинстве столбцов хоть какая то доля пропущена, значит нужно будет все их заполнить


### Преобразуем данные. Столбец с датой преобразуем только к месяцу, столбец с информацией о дожде завтра преобразуем аналогично RainTomorrow




In [615]:
X.RainToday = X.RainToday.replace({'No':0, 'Yes': 1})

X['Date']=[int(x[5:7]) for x in X['Date']]


### В текстовых данных заполним пропуски на их моды, в численных данных - медианой

In [616]:
categorial = list(X.select_dtypes(include = ['object']).columns.values)
for col in categorial:
    if(X[col].isnull().values.any()):
        X[col].fillna(X[col].mode()[0],inplace=True)
numerical=list(X.select_dtypes(include = ['int64', 'float64']).columns.values)
for col in numerical:
    if(X[col].isnull().values.any()):
        X[col].fillna(X[col].median(),inplace=True)

### Указания навправлений ветра преобразуем к градусам (как в компасе) 

In [617]:
dic={'N':0,'W':270,'E':90,'S':180,'SE':135,'SW':225,'NE':45,'NW':315,'NNE':22,'ENE':67, 'ESE':112,'SSE':157,'SSW':202,'WSW':247,'WNW':292,'NNW':337}

X['WindGustDir']=[dic[x] for x in X['WindGustDir']]
X['WindDir9am']=[dic[x] for x in X['WindDir9am']]
X['WindDir3pm']=[dic[x] for x in X['WindDir3pm']]


### Проведем hashing trick для столбца Location в котором 49 уникальных позиций в данном случае параметр n_features = 10

In [618]:
from sklearn.feature_extraction import FeatureHasher
hasher = FeatureHasher(n_features=10, input_type='string')

hashed_features = hasher.transform(X.values.astype(str))

hashed_df = pd.DataFrame(hashed_features.toarray())

hashed_df.columns = ['hashed_feature' + str(i) for i in range(hashed_df.shape[1])]

X = pd.concat([X, hashed_df], axis=1)
del X['Location']



### Применим MinMaxScaler

In [619]:
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

X.describe()


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
count,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,...,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000
mean,0.491140,0.487867,0.529780,0.006272,0.035746,0.554637,0.523231,0.262947,0.450314,0.506978,...,0.471464,0.487268,0.443430,0.477307,0.458335,0.506308,0.438609,0.476472,0.539657,0.439414
std,0.311501,0.150682,0.134397,0.022713,0.021968,0.191009,0.298452,0.101865,0.324751,0.296091,...,0.103959,0.114946,0.102785,0.104890,0.113159,0.102356,0.105708,0.113834,0.107677,0.097387
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.181818,0.379717,0.429112,0.000000,0.027586,0.565517,0.267062,0.193798,0.133531,0.267062,...,0.375000,0.416667,0.416667,0.437500,0.357143,0.416667,0.384615,0.428571,0.454545,0.357143
50%,0.454545,0.483491,0.517958,0.000000,0.033103,0.586207,0.534125,0.255814,0.465875,0.465875,...,0.500000,0.500000,0.416667,0.500000,0.428571,0.500000,0.461538,0.500000,0.545455,0.428571
75%,0.727273,0.596698,0.623819,0.001617,0.037241,0.600000,0.801187,0.310078,0.732938,0.732938,...,0.562500,0.583333,0.500000,0.562500,0.500000,0.583333,0.461538,0.571429,0.636364,0.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Случайный лес для выявления важности каждого из признаков и возможного отбрасывания их

In [620]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import recall_score
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,shuffle = True)

# rfc = RandomForestClassifier()
# rfc.fit(X_train, y_train)
# display(rfc.score(X_train, y_train))

# # 1.0

### График важности признаков

In [621]:
# feats = {}
# for feature, importance in zip(X.columns, rfc.feature_importances_):
#     feats[feature] = importance
# importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-Importance'})
# importances = importances.sort_values(by='Gini-Importance', ascending=False)
# importances = importances.reset_index()
# importances = importances.rename(columns={'index': 'Features'})
# sns.set(font_scale = 5)
# sns.set(style="whitegrid", color_codes=True, font_scale = 1.7)
# fig, ax = plt.subplots()
# fig.set_size_inches(12,8)
# sns.barplot(x=importances['Gini-Importance'], y=importances['Features'], data=importances, color='skyblue')
# plt.xlabel('Importance', fontsize=25, weight = 'bold')
# plt.ylabel('Features', fontsize=25, weight = 'bold')
# plt.title('Feature Importance', fontsize=25, weight = 'bold')
# display(plt.show())
# display(importances)

### Трюк с Location не привел ни к чему хорошему, так как по итогу все hashed_feature не сильно повлиляли на итоговый результат и можно их всех удалить

In [622]:
del X[30]
del X[29]
del X[28]
del X[27]
del X[26]
del X[25]
del X[24]
del X[23]
del X[22]
del X[21]




### PCA - метод главных компонент

In [623]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
pca_test = PCA(n_components=21)
pca_test.fit(X_train)
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.head(21))

,Cumulative Variance Ratio,Explained Variance Ratio
0,0.255742,0.255742
1,0.420739,0.164996
2,0.536776,0.116037
3,0.645130,0.108354
4,0.735228,0.090097
5,0.811519,0.076291
6,0.857227,0.045709
7,0.889802,0.032574
8,0.919457,0.029656
9,0.941748,0.022290


### Видим что суммарная дисперсия перестает сильно меняться после 13 признака, поэтому можно отсечь остальные

In [624]:
pca = PCA(n_components=13)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

### Разделение выборки

In [625]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,shuffle = True)


### Логистическая регрессия

In [626]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
print(accuracy_score(clf.predict(X_test),y_test))


0.8403330614081972


### Метод ближайших соседей

In [628]:

knn = KNeighborsClassifier(3).fit(X_train, y_train)
print(accuracy_score(knn.predict(X_test),y_test))


AttributeError: 'Flags' object has no attribute 'c_contiguous'

### Наивный байессовский классификатор

In [ ]:
gnb = GaussianNB().fit(X_train, y_train)
print(accuracy_score(gnb.predict(X_test),y_test))

In [ ]:
X.info()

### Реализация логистической регрессии
__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [ ]:
theta = np.array([1, 2, 3])

X =  np.array([[ 1,  1, 1],
               [-1, -2, 1],
               [-1, -2, 2],
               [-2, -2, -3]
              ])

y = np.array([1, 1, 0, 0])

In [ ]:
def probability(theta, X):
    result = 1/(1+np.exp(-np.dot(X,theta)))
    return result
prob = probability(theta, X)


assert type(prob) == np.ndarray, 'Возвращается неверный тип'
assert prob.shape == (X.shape[0],), 'Неверный размер массива'
assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [ ]:
def binary_class_prediction(theta, X, threshold =.5):
    prob =  probability(theta, X)
    result = np.where(prob<0.5,0,1)
    return result

y_pred = binary_class_prediction(theta, X)


assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'
assert min(y_pred) == 0, 'Функция считается неверно'
assert max(y_pred) == 1, 'Функция считается неверно'

__Функционал качества логистической регрессии__

Запишем правдободовие выборки для меток класса $y \in \{+1, -1\}$ 

$$Likelihood(a, X^\ell) = \prod_{i = 1}^{\ell} a(x_i,\theta)^{[y_i = +1]} (1 - a(x_i, \theta))^{[y_i = -1]} → \operatorname*{max}_{\theta}$$ 

Прологарифмируем правдоподобие выборки и перейдем к задаче минимизации:

$$Q(a, X^\ell) =     -\sum_{i = 1}^{\ell} 
        [y_i = +1] \log a(x_i, \theta)
        +
        [y_i = -1] \log (1 - a(x_i, \theta)) \to \operatorname*{min}_{\theta}$$ 
        
Подставим $a(x, \theta)$ в функцинал качества:

$$ Q(a, X^\ell) = -\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{\exp(-\langle \theta, x_i \rangle)}{1 + \exp(-\langle \theta, x_i \rangle)}
\right)
=\\
=
-\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{1}{1 + \exp(\langle \theta, x_i \rangle)}
\right)
=\\
=
\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) $$
    

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:

In [ ]:
def logloss(theta, X, y): 
    y_temp=np.where(y==0,-1,1)
    result = (1/len(y))*(np.sum(np.log(1+np.exp(-y_temp*np.dot(X,theta)))))
    print(result)
    return result
logloss(theta,X,y)

In [ ]:
assert logloss(theta, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

<b>Вход: </b> Выборка $X^\ell$, темп обучения $h$

<b>Выход: </b> оптимальный вектор весов $\theta$

1.  Инициализировать веса $\theta$
2.  Инициализировать оценку функционала качества: $Q(a, X^\ell)$
3.  <b>Повторять</b>: 

    Выбрать случайным образом подвыборку объектов $X^{batch} =\{x_1, \dots,x_n \}$ из $X^{\ell}$
    
    Рассчитать градиент функционала качества: $\nabla Q(X^{batch}, \theta)$
    
    Обновить веса: $\theta := \theta - h\cdot \nabla Q(X^{batch}, \theta)$
       
    <b>Пока</b> значение $Q$ и/или веса $\theta$ не сойдутся   

Реализуем функцию рассчета градиента функционала качества

$$\frac{\partial Q(a, X^{batch}) }{\partial \theta_j}   = \frac{\partial \frac{1}{n}\sum_{i = 1}^{n}
    \log \left(
        1 + \exp(- y_i \langle \theta, x_i \rangle)
    \right)} {\partial \theta_j}  = \frac{1}{n}\sum_{i = 1}^{n}
     \frac {1}{
        1 + \exp(- y_i \langle \theta, x_i \rangle)} \cdot  \exp(- y_i \langle \theta, x_i \rangle) \cdot -y_i x_{ij}$$

Реализуйте рассчет градиента в матричном виде:

In [ ]:
def gradient(theta, X, y):
    y_pred=binary_class_prediction(theta, X)
    n=X.shape[0]
    result=np.dot(X.T, (y_pred - y)) / n
    
    return result 

assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'

Функция обучения уже реализована

In [ ]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)
    
    errors = []
    
    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
    # plt 
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')
        
        
    for _ in range(iters):  
        
        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        # считаем производные
        grad = gradient(theta, X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'
        
        
        # Обновляем веса
        
        theta -= grad * h
        
        theta_history = np.vstack((theta_history, theta))
        
        # error
        loss = logloss(theta, X, y)
        errors.append(loss)
        
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')
            
            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()   
            
    return theta

In [ ]:
X, y = make_classification(n_samples=2000)

In [ ]:
optimal_theta = fit(X, y)

In [ ]:
y_pred = binary_class_prediction(optimal_theta, X)